In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [23]:
def FNS(scores):
    domination = (scores[:, None, :] <= scores[None, :, :]).all(2) # domination[i, j] = "i dominuje j"
    domination &= ~(scores[:, None, :] == scores[None, :, :]).all(2)
    Nx = domination.sum(0)
    
    Pf = []
    ranks = np.zeros(scores.shape[0])
    r = 0
    Q = np.argwhere(Nx == 0)
    while Q.size > 0:
        Q = Q[:, 0]
        Nx[Q] = -1
        Pf.append(Q)
        ranks[Q] = r
        r += 1
        for i in Q:
            Nx[domination[i, :]] -= 1
        Q = np.argwhere(Nx == 0)
        
    return Pf, ranks

def crowding_distance(scores):
    indices = np.argsort(scores, 0)
    sorted_scores = np.take_along_axis(scores, indices, 0)
    cd = np.zeros(scores.shape[0])
    for k in range(scores.shape[1]):
        cd[indices[[0, -1], k]] = np.inf
        cd[indices[1:-1, k]] += (sorted_scores[2:, k] - sorted_scores[:-2, k]) / (sorted_scores[-1, k] - sorted_scores[0, k])
    return cd

def random_population(d, n, x_min, x_max):
    return np.hstack([np.random.uniform(x_min, x_max, (n, d))])

def tournament_selection(ranks, dists, n):
    candidates = np.random.choice(n, (n, 2), replace=True)
    mask = np.where(
        ranks[candidates[:, 0]] == ranks[candidates[:, 1]],
        dists[candidates[:, 0]] > dists[candidates[:, 1]],
        ranks[candidates[:, 0]] < ranks[candidates[:, 1]]
    )
    result = candidates[:, 1]
    result[mask] = candidates[mask, 0]
    return result

def crossover(x, p, eta): # simulated binary crossover
    n, d = x.shape
    l = n // 2
    mask = np.random.random(l) <= p
    m = np.sum(mask)
    mi = np.random.random((m, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)),
        np.power(1. / (2.*(1-mi)), 1. / (eta+1.))
    )
    c1 = x[:l, :].copy()
    c2 = x[l:, :].copy()
    c1[mask, :] = 0.5 * (1 + beta) * x[:l, :][mask, :] + 0.5 * (1 - beta) * x[l:, :][mask, :]
    c2[mask, :] = 0.5 * (1 + beta) * x[:l, :][mask, :] + 0.5 * (1 - beta) * x[l:, :][mask, :]
    return np.vstack([c1, c2])

def mutation(x, x_min, x_max, p, eta): # polynomial mutation
    n, d = x.shape
    mask = np.random.random(x.shape[0]) <= p
    m = np.sum(mask)
    mi = np.random.random((m, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)) - 1.,
        1. - np.power(2.*(1-mi), 1. / (eta+1.))
    )
    y = x.copy()
    y[mask, :] = np.where(
        mi < 0.5,
        x[mask, :] + beta * (x[mask, :] - x_min),
        x[mask, :] + beta * (x_max - x[mask, :])
    )
    return y

def elitist_selection(fronts, dists, to_take):
    taken = []
    for front in fronts:
        if len(front) <= to_take:
            taken += list(front)
            if len(front) == to_take:
                break
            to_take -= len(front)
        else:
            indices = np.argsort(-dists[front])[:to_take]
            taken += list(indices)
            break
    return taken

def constraint_violation(constraints):
    n, d = constraints.shape
    sort_indices = np.argsort(constraints, 0)
    violations = np.zeros(n)
    for i in range(d):
        values, counts = np.unique(constraints[:, i], return_counts=True) # unikalne wartości są zwracane posortowane
        counts = np.cumsum(counts)
        counts = list(counts)
        if values[0] != 0:
            counts = [0] + counts
        for rank, (j, k) in enumerate(zip([0] + counts, counts + [len(counts)])):
            violations[sort_indices[j:k, i]] += rank
    return violations
        

def IDEA(objective, n_constraints, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=10):
    n_f = n - n_inf
    population = random_population(d, n, x_min, x_max)
    populations = [population.copy()]
    obj_results = objective(population)
    constraint_values = obj_results[:, -n_constraints:]
    violation_measure = constraint_violation(constraint_values)
    scores = np.concatenate([obj_results[:, :-n_constraints], violation_measure[:, None]], 1)
    scores_hist = [scores.copy()]

    fronts, ranks = FNS(scores)
    dists = crowding_distance(scores)
    
    for iter_ in range(num_iterations):
        parent_indices = tournament_selection(ranks, dists, n)
        offspring = crossover(population[parent_indices, :], p_c, eta_c)
        offspring = np.clip(offspring, x_min, x_max)
        offspring = mutation(offspring, x_min, x_max, p_m, eta_m)
        
        offspring_obj_results = objective(offspring)
        offspring_constraint_values = offspring_obj_results[:, -n_constraints:]
        offspring_violation_measure = constraint_violation(offspring_constraint_values)
        offspring_scores = np.concatenate([offspring_obj_results[:, :-n_constraints], offspring_violation_measure[:, None]], 1)
        
        population = np.vstack([population, offspring])
        scores = np.vstack([scores, offspring_scores])
        
        dists = crowding_distance(scores)
        mask_f = scores[:, -1] == 0
        mask_inf = ~mask_f
        s_f = np.sum(mask_f)
        s_inf = np.sum(mask_inf)
        if s_f < n_f:
            to_take_f = s_f
            to_take_inf = n - s_f
        elif s_inf < n_inf:
            to_take_inf = s_inf
            to_take_f = n - s_inf
        else:
            to_take_f = n_f
            to_take_inf = n_inf
            
        population_f = population[mask_f, :]
        scores_f = scores[mask_f, :]
        dists_f = dists[mask_f]
        fronts, ranks = FNS(population_f)
        taken_f = elitist_selection(fronts, dists_f, to_take_f)
        
        population_inf = population[mask_inf]
        scores_inf = scores[mask_inf, :]
        dists_inf = dists[mask_inf]
        fronts, ranks = FNS(population_inf)
        taken_inf = elitist_selection(fronts, dists_inf, to_take_inf)
        
        population = np.vstack([population_f[taken_f, :], population_inf[taken_inf, :]])
        scores = np.vstack([scores_f[taken_f, :], scores_inf[taken_inf, :]])
        dists = np.hstack([dists_f[taken_f], dists_inf[taken_inf]])
        fronts, ranks = FNS(population)
        
        populations.append(population.copy())
        scores_hist.append(scores.copy())
        
        if iter_ % log_interval == 0:
            print(f"Iteration {iter_}, #feasible: {to_take_f}, #infeasible: {to_take_inf}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    print(f"Iteration {iter_}, #feasible: {to_take_f}, #infeasible: {to_take_inf}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    return np.stack(populations, 0), np.stack(scores_hist, 0)

In [30]:
def g1_objective(x):
    c = np.stack([
        # objective
        5 * x[:, :4].sum(1) - 5 * (x[:, :4]**2).sum(1) - x[:, 4:].sum(1),
        # constraints
        2*x[:, 0] + 2*x[:, 1] + x[:, 9] + x[:, 10] - 10.,
        2*x[:, 0] + 2*x[:, 2] + x[:, 9] + x[:, 11] - 10.,
        2*x[:, 1] + 2*x[:, 2] + x[:, 10] + x[:, 11] - 10.,
        -8*x[:, 0] + x[:, 9],
        -8*x[:, 1] + x[:, 10],
        -8*x[:, 2] + x[:, 11],
        -2*x[:, 3] - x[:, 4] + x[:, 9],
        -2*x[:, 5] - x[:, 6] + x[:, 10],
        -2*x[:, 7] - x[:, 8] + x[:, 11],
    ], 1)
    c[:, 1:] = np.maximum(c[:, 1:], 0.)
    return c
    

def g6_objective(x):
    c = np.stack([
        # objective
        (x[:, 0] - 10.)**3 + (x[:, 1] - 20.)**3,
        # constraints
        -(x[:, 0] - 5)**2 - (x[:, 1]-5)**2 + 100,
        (x[:, 0] - 6)**2 + (x[:, 1]-5)**2 - 82.81,
    ], 1)
    c[:, 1:] = np.maximum(c[:, 1:], 0.)
    return c


def g2_objective(x):
    n, d = x.shape
    c = np.cos(x)
    r = np.stack([
        # objective
        -np.abs(((c**4).sum(1) - 2*(c**2).prod(1)) / np.sqrt((np.arange(d) * x**2).sum(1))),
        # constraints
        0.75 - x.prod(1),
        x.sum(1) - 7.5*d
    ], 1)
    r[:, 1:] = np.maximum(r[:, 1:], 0.)
    return r

In [25]:
objective = g6_objective
n_constraints = 2

x_min = np.array([13., 0.])
x_max = 100.
d = 2
n = 200
n_inf = int(0.2*n)
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, n_constraints, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, #infeasible: 200, scores: [-3.41338854e+03  1.00000000e+00] [1.29683721e+05 5.09550000e+01] [7.11398873e+05 1.54000000e+02]
Iteration 100, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-6.99740252e+03  3.73000000e+00] [-990.13120101   37.        ]
Iteration 200, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.11881067e+03  5.20000000e+00] [-6820.64224919    42.        ]
Iteration 300, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.03523124e+03  3.84500000e+00] [-1409.40217897    39.        ]
Iteration 400, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.1605726e+03  5.1350000e+00] [-6957.44719925    39.        ]
Iteration 500, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.05796952e+03  5.30500000e+00] [-4814.44555977    40.        ]
Iteration 600, #feasible: 160, #infeasible: 40, scores: [-7973.     0.] [-7.02706373e+03  4.51500000e+00] [245.8657743  40.       ]
Iteration 700, #feasible: 160, 

In [26]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-6961.687051583694

In [27]:
objective = g1_objective
n_constraints = 9

x_min = 0.
x_max = np.ones(13)
x_max[9:12] = 100.
d = 13
n = 600
n_inf = int(0.2*n)
eta_c = 5.
eta_m = 20.
p_c = 0.9
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, n_constraints, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, #infeasible: 600, scores: [-276.46475371  111.        ] [-152.16365149 2277.74666667] [ -16.20226566 4944.        ]
Iteration 100, #feasible: 480, #infeasible: 120, scores: [-300.71570915    0.        ] [-33.773287   215.86833333] [3.0466555e+00 4.1240000e+03]
Iteration 200, #feasible: 480, #infeasible: 120, scores: [-300.49136323    0.        ] [-32.03441701 193.235     ] [3.3991573e+00 4.1240000e+03]
Iteration 300, #feasible: 480, #infeasible: 120, scores: [-301.1465693    0.       ] [-23.07003418 127.59833333] [3.41675105e+00 4.12400000e+03]
Iteration 400, #feasible: 480, #infeasible: 120, scores: [-293.13726401    0.        ] [-14.15785696  82.74      ] [   4.14342603 1110.        ]
Iteration 500, #feasible: 480, #infeasible: 120, scores: [-301.26332154    0.        ] [-26.15875237 120.205     ] [   4.63830932 1471.        ]
Iteration 600, #feasible: 480, #infeasible: 120, scores: [-300.07224632    0.        ] [-17.32567651  75.73      ] [   4.8223892 152

In [28]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-7.973440774316874

In [31]:
objective = g2_objective
n_constraints = 2

x_min = 1e-4
x_max = 10.
d = 20
n = 400
n_inf = int(0.2*n)
eta_c = 4.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, n_constraints, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

/home/i291318/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/home/i291318/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in greater


Iteration 0, #feasible: 396, #infeasible: 4, scores: [-0.21426255  0.        ] [-0.10794033  0.015     ] [-0.0394651  2.       ]
Iteration 100, #feasible: 320, #infeasible: 80, scores: [-0.38791929  0.        ] [-0.16865374 10.9625    ] [-4.90829889e-03  1.07000000e+02]
Iteration 200, #feasible: 320, #infeasible: 80, scores: [-0.47836362  0.        ] [-0.2085008 11.7625   ] [-3.08070701e-03  1.10000000e+02]
Iteration 300, #feasible: 320, #infeasible: 80, scores: [-0.56055817  0.        ] [-0.23875132 11.82      ] [-2.45502702e-03  1.10000000e+02]
Iteration 400, #feasible: 320, #infeasible: 80, scores: [-0.71007635  0.        ] [-0.24891758 11.62      ] [-1.91831451e-03  1.10000000e+02]
Iteration 500, #feasible: 320, #infeasible: 80, scores: [-1.03265005  0.        ] [-0.29203451 12.5175    ] [-1.3264931e-03  1.1000000e+02]
Iteration 600, #feasible: 320, #infeasible: 80, scores: [-4.87221115  0.        ] [-0.62085383 11.4025    ] [-1.29697338e-03  1.10000000e+02]
Iteration 700, #feasibl

In [32]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-0.7765625669776021

Sanity check

In [33]:
i = scores[-1, scores[-1, :, 1] == 0., 0].argmin()
i

312

In [34]:
x = populations[-1][scores[-1, :, 1] == 0., :][[i], :]

In [35]:
objective(x)

array([[-0.77656257,  0.        ,  0.        ]])